# Run a Machine Psychophysics Experiment Signal Detection Theory -- no threshold

In [1]:
## load requirements
import sys, os, distutils.core
import torch, detectron2
import cv2
import matplotlib.pyplot as plt

from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import argparse
import pandas as pd

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
import detectron2.structures.boxes as boxes

#import coco json ID to label conversion
import coco_object_categories

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pickle

device = 'cuda:0'

######CHANGE THIS BASED ON YOUR FILE LOCATIONS!!!
base_dir = '.'

In [2]:
#the normal nnotation file in coco has 90 categories for some unknown reason. Use the real ones from this text file.
anns_file = './train_finetune/coco80annotations.txt'
with open(anns_file) as f:
    anns = [line.replace('\n','') for line in f.readlines()]

label_converter_dict = {}
for i, ann in enumerate(anns):
    label_converter_dict[ann] = i #convert to zero-indexed labels
#label_converter_dict

In [3]:
## Load In Mongrel Pairs

## read in data info
bbx_list = pd.read_csv('./psychophysics_experiment/cocop_bbx_fixations_new.csv')

#imlist = list(set(list(bbx_list['image_name'])))
#imlist = ['000000009590.jpg','000000529148.jpg','000000545100.jpg','000000255165.jpg','000000322163.jpg'] #'000000311002.jpg', <- fix this one, it's 10 mongrels of the mask
imlist = ['000000011197.jpg','000000009769.jpg','000000009590.jpg','000000011197.jpg','000000016439.jpg',
'000000018150.jpg','000000018380.jpg','000000019042.jpg',
'000000061268.jpg','000000063602.jpg','000000067616.jpg','000000119445.jpg',
'000000119516.jpg','000000186422.jpg','000000203864.jpg',
'000000209530.jpg','000000222299.jpg','000000226417.jpg','000000255165.jpg','000000281409.jpg',
'000000311002.jpg','000000322163.jpg',
'000000484351.jpg','000000491008.jpg','000000509258.jpg','000000529148.jpg','000000545100.jpg']
print(imlist)

eccs_px = [0,80,160,240,320] 
eccs_dg = [0,5,10,15,20]

#imfolder = f'~/RosenholtzLab_shared/uniform_absent_exp_stims/ecc_{eccpx}/mongrel_{imname}_inpaint_ecc_{eccpx}_{n}.jpg' #present/absent/uni_{eccdg}/mongrel_{imname}_ecc_{eccdg}.jpg
#imfolder = f'~/RosenholtzLab_shared/detection_repos/detrex/cocop_stims/' #present/absent/uni_{eccdg}/mongrel_{imname}_ecc_{eccdg}.jpg
#imfolder = '~/RosenholtzLab_shared/cocop_stims3_scoring/uniform/'
#absent/samples/ecc320/mongrel_000000545100_ecc_320_4.jpg
#imfolder_present = f'~/RosenholtzLab_shared/final_experiment_mulitple_mongrel_images_present/'
#imfolder_absent = f'~/RosenholtzLab_shared/final_experiment_mulitple_mongrel_images_present/'
imfolder_present = '/home/gridsan/groups/RosenholtzLab/final_experiment_multiple_mongrel_images_present/'
imfolder_absent = '/home/gridsan/groups/RosenholtzLab/final_experiment_multiple_mongrel_images_absent/'
#imfolder_present = imfolder_absent = imfolder = '/home/gridsan/groups/RosenholtzLab/cocop_stims3_scoring/uniform/'


['000000011197.jpg', '000000009769.jpg', '000000009590.jpg', '000000011197.jpg', '000000016439.jpg', '000000018150.jpg', '000000018380.jpg', '000000019042.jpg', '000000061268.jpg', '000000063602.jpg', '000000067616.jpg', '000000119445.jpg', '000000119516.jpg', '000000186422.jpg', '000000203864.jpg', '000000209530.jpg', '000000222299.jpg', '000000226417.jpg', '000000255165.jpg', '000000281409.jpg', '000000311002.jpg', '000000322163.jpg', '000000484351.jpg', '000000491008.jpg', '000000509258.jpg', '000000529148.jpg', '000000545100.jpg']


In [4]:
# Load in Model Stuff
def create_predictor(ecc,thresh):
    #baseline model
    cfg = get_cfg()
    # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
    #cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    #cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
    #cfg.merge_from_file('./detectron2/configs/quick_schedules/mask_rcnn_R_50_FPN_inference_acc_test.yaml')
    cfg.merge_from_file('./detectron2/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml')
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = thresh #0.01  # set threshold for this model
    #cfg.MODEL.RETINANET.SCORE_THRESH_TEST = thresh
    # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
    #cfg.MODEL.WEIGHTS = f'/{base_dir}detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl'
    if(ecc=='-2'):
        print('scratchtrain model')        
        cfg.MODEL.WEIGHTS = f'/{base_dir}/output_trainscratch/trainscratch_ecc100/model_final.pth'

    elif(ecc=='-1'):
        #baseline
        print('baseline model')
        cfg.MODEL.WEIGHTS = f'/{base_dir}/detectron2/model_weights/R_50_FPN_3x/model_final_280758.pkl'
    # elif(ecc=='100'):
    #     cfg.MODEL.WEIGHTS = f'/{base_dir}/finetune_ecc{ecc}/model_final.pth'
    else:
        cfg.MODEL.WEIGHTS = f'/{base_dir}/finetune_ecc{ecc}/model_final.pth'
    #cfg.SOLVER.BASE_LR = 0.0001 #0.001 
    #model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    
    # pass in the bounding box
    cfg.MODEL.PROPOSAL_GENERATOR.NAME = "PrecomputedProposals"
    cfg.freeze()
    predictor = DefaultPredictor(cfg)
    return(cfg, predictor)



In [5]:
def gen_mong_lists(imname,ecc):
    imname = imname.replace('.jpg','')

    present_mongs = []
    absent_mongs = []
    #present & absent
    if ecc==0:
        nmongs = 1
    else:
        nmongs = 10
    
    for i in range(nmongs):
        try:
            if(ecc==0):
                pmong = cv2.imread(os.path.join(imfolder_present,f'ecc{ecc}/{imname}.jpg'))
            else:
                pmong = cv2.imread(os.path.join(imfolder_present,f'ecc{ecc}/mongrel_{imname}_ecc_{ecc}_{i}.jpg'))
            present_mongs.append(pmong)
        except:
            print(os.path.join(imfolder_present,f'ecc{ecc}/mongrel_{imname}_ecc_{ecc}_{i}.jpg'))
        try:
            if(ecc==0):
                among = cv2.imread(os.path.join(imfolder_absent,f'ecc{ecc}/{imname}.png'))
            else:                
                among = cv2.imread(os.path.join(imfolder_absent,f'ecc{ecc}/mongrel_{imname}_ecc_{ecc}_{i}.jpg'))
            #among = cv2.cvtColor(among, cv2.COLOR_BGR2RGB)
            absent_mongs.append(among)
        except:
            print(os.path.join(imfolder_absent,f'ecc{ecc}/mongrel_{imname}_ecc_{ecc}_{i}.jpg'))

        
    return(present_mongs, absent_mongs)

#present_mongs, absent_mongs = gen_mong_lists(imlist[2],160)

In [6]:
def cropbbxtoimage(bbx,imsz):
    '''
    If bounding box extends outside image size, crop it.
    '''
    x,y,w,h = bbx
    #print(imsz)
    imx,imy,_ = imsz
    x=0 if x<0 else x #make sure x isn't negative
    y=0 if y<0 else y #make sure y isn't negative
    w=imx if w+x>imx else w #make sure width isn't larger than image
    h=imy if h+y>imy else h #make sure height isn't taller than image
    return([x,y,w,h])

def bbx_xywhtoxyxy(bbx):
    x,y,w,h = bbx
    x2 = x + w
    y2 = y + h
    return([x,y,x2,y2])

def bbx_xyxytoxywh(bbx):
    x,y,x2,y2 = bbx
    w = x2-x
    h = y2-y
    return([x,y,w,h])


In [7]:
def get_prediction_match(model_predictions,
                         ground_truth_object_id,
                         ground_truth_bbx,
                         padded_ground_truth_bbx,
                         mong=None,
                        enforce_category=True):
    #find where ground truth object identity matches model class predictions
    model_class_predictions = np.array(model_predictions['instances'].pred_classes.cpu())
    
    if(enforce_category):
        prediction_matches = model_predictions['instances'][model_predictions['instances'].pred_classes==ground_truth_object_id]
    else:
        prediction_matches = model_predictions['instances']

    if len(prediction_matches) == 0:
        return(0.)
    #otherwise find the best match
    else:

        best_match_pred = 0.
        #loop through matches and return the highest one that operlaps
        for i in range(len(prediction_matches)):
            pred = prediction_matches[i]
            model_bbx_predictions = pred.pred_boxes
           
            ioa = boxes.pairwise_ioa(padded_ground_truth_bbx, model_bbx_predictions)
            ioa = np.array(ioa.cpu().numpy())[0][0]
            
            #print('iou is',iou)
            #condition 1
            if ioa > 0.75:
                #condition 2
                areas_ratio = ground_truth_bbx.area()/model_bbx_predictions.area()
                if (areas_ratio > 0.5) and (areas_ratio < 2.): #0.6/1.67 #0.75/1.33 #0.9/1.11 #0.5/2
                    score = pred.scores.cpu().numpy()[0]
                    #print(score)
                    #if thats our best score, keep it
                    if score > best_match_pred:
                        #print('updating score to',score)
                        best_match_pred = score
        #print('best',best_match_pred)
        if mong is not None:
            plt.show()

    #if more than one report the highest
    return(best_match_pred)

In [8]:
def get_prediction_matches_no_threshold(model_predictions,
                                       ground_truth_object_id):
    model_class_predictions = model_predictions['instances'].pred_classes.detach().cpu()
    model_scores = model_predictions['instances'].scores.detach().cpu()
    
    target_class_idx = (model_class_predictions == ground_truth_object_id).nonzero(as_tuple=True)[0].item()
    return model_scores[target_class_idx].item()
    

In [9]:
def sample_probabilities(pmong_prob, among_prob, nsamples):

    if(pmong_prob==0. and among_prob==0.):
        norm_among_prob = 0.5
    elif(pmong_prob==0.):
        norm_among_prob = 1.
    elif(among_prob==0.):
        norm_among_prob = 0.
    else:
        norm_among_prob = among_prob / (pmong_prob + among_prob)
    #print(norm_among_prob)
    samples = [np.random.rand() for i in range(nsamples)]
    #if random [0,1] is greater than norm_among_prob then it's correct
    #print(samples)
    #print(norm_among_prob)
    correct_preds = 1*np.greater(samples,norm_among_prob)
    #print(correct_preds)
    acc = np.mean(correct_preds)
    return(acc)

In [10]:
from scipy.stats import norm
import numpy as np

def sample_probabilities_sdt(pmong_prob, among_prob, nsamples):
    
    #probability can't be exacly zero
    pmong_prob = pmong_prob if pmong_prob>=1e-5 else 1e-5
    among_prob = among_prob if among_prob>=1e-5 else 1e-5

    #calcluate z scores
    among_z = norm.ppf(q=(among_prob)) #ppd takes lower tail probability, among_prob is upper.
    pmong_z = norm.ppf(q=(pmong_prob)) #ppd takes lower tail probability, pmong_prob is upper.
    
    d_prime = pmong_z - among_z #d prime is difference beween the two
    
    #print('among_z: ',among_z)
    #print('pmong_z: ',pmong_z)
    #print('d_prime: ',d_prime)
    
    among_dist_samples = np.random.normal(loc=0,size=nsamples) #sample from normal for among dist
    pmong_dist_samples = np.random.normal(loc=d_prime,size=nsamples) #sample from normal for pmong dist
    
    #print('absent samples: ',among_dist_samples)
    #print('present samples: ',pmong_dist_samples)
    correct = 1*np.greater(pmong_dist_samples, among_dist_samples) #check if pmong was higher
    acc = np.mean(correct)
    return(acc)

In [ ]:
model_names = ['-1','0','-2','100','101','5','10','15','20']
#finished: '100','-1',
#model_names = ['101']#,'-2','5','10','15']
nsamples = 1000
pred_thresh = 0.0 # 0.1
category_match_bool = False

accuracies = {}
for mi, model_name in enumerate(model_names):
    accuracies[model_name] = {}
    #create predictor
    cfg, predictor = create_predictor(model_name,thresh=pred_thresh)
    for im in imlist:
        #make a dictionary
        accuracies[model_name][im] = {}
        for ecci, ecc in enumerate(eccs_px):
            accuracies[model_name][im][ecc] = {}
            present_mongs, absent_mongs = gen_mong_lists(im,ecc)
            #store accuracy array for pairs of this mongrel + eccentricity combo
            im_ecc_accuracies = []

            ###### Run Inference on model (in parallel)
            #pmong_model_predictions_batch = predictor(torch.from_numpy(np.array(present_mongs)))
            #among_model_predictions_batch = predictor(torch.from_numpy(np.array(absent_mongs)))
            
            #loop through pairs            
            for i in range(len(present_mongs)):
                for j in range(len(absent_mongs)):
                    #print(i,j)
                    pmong = present_mongs[i]
                    among = absent_mongs[j]
                    
                    # manually apply image pre-processing
                    import detectron2.data.transforms as T
                    with torch.no_grad():  # https://github.com/sphinx-doc/sphinx/issues/4258
                        pheight, pwidth = pmong.shape[:2]
                        aheight, awidth = among.shape[:2]
                        # aug = T.ResizeShortestEdge(
                        #         [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST
                        #     )
                        # image = aug.get_transform(original_image).apply_image(original_image)
                        pmong = torch.as_tensor(pmong.astype("float32").transpose(2, 0, 1))
                        among = torch.as_tensor(among.astype("float32").transpose(2, 0, 1))
                        pinputs = {"image": pmong, "height": pheight, "width": pwidth}
                        ainputs = {"image": among, "height": aheight, "width": awidth}   

                    ##### Get Ground Truth
                    #get ground truth bounding box
                    ground_truth_bbx = [bbx_list[bbx_list['image_name']==im]['bbx_x_16'].item(),
                                       bbx_list[bbx_list['image_name']==im]['bbx_y_16'].item(),
                                       bbx_list[bbx_list['image_name']==im]['bbx_w_16'].item(),
                                       bbx_list[bbx_list['image_name']==im]['bbx_h_16'].item()]
                    #padded_ground_truth_bbx = ground_truth_bbx
                    padded_ground_truth_bbx = [sum(x) for x in zip(ground_truth_bbx, [-ecc//4, #move x half a pooling region to left
                                                                                  -ecc//4, #move y half a pooling region to right
                                                                                   ecc//2, #increase width by full pooling region
                                                                                   ecc//2] #increase height by full pooling region
                                       )]
                    #crop bbx if it extends beyond edge of image
                    padded_ground_truth_bbx = cropbbxtoimage(ground_truth_bbx,pmong.shape)
                    #print(print('bbx before',ground_truth_bbx))
                    padded_ground_truth_bbx = bbx_xywhtoxyxy(padded_ground_truth_bbx)
                    #print(print('bbx after',ground_truth_bbx))
                    #convert to detectron2 Boxes for IOU calculation
                    padded_ground_truth_bbx = boxes.Boxes(torch.unsqueeze(torch.Tensor(padded_ground_truth_bbx),0).to(device))

                    #convert original ground truth bounding box also
                    ground_truth_bbx = bbx_xywhtoxyxy(ground_truth_bbx)
                    ground_truth_bbx = boxes.Boxes(torch.unsqueeze(torch.Tensor(ground_truth_bbx),0).to(device))
                    
                    #ground truth object category & ID
                    ground_truth_object_name = bbx_list[bbx_list['image_name']==im]['object_name'].item()
                    ground_truth_object_id = label_converter_dict[ground_truth_object_name]
                    
                    # fix a proposal bounding box
                    from detectron2.structures import Boxes, Instances
                    pres = Instances((pheight, pwidth))
                    pres.proposal_boxes = padded_ground_truth_bbx
                    pproposals = pres
                    
                    ares = Instances((aheight, awidth))
                    ares.proposal_boxes = padded_ground_truth_bbx
                    aproposals = ares

                    # add box proposal to predictor inputs
                    pinputs['proposals'] = pproposals
                    ainputs['proposals'] = aproposals
                    
                    
                    
                    pmong_model_predictions = predictor.model([pinputs])[0]
                    among_model_predictions = predictor.model([ainputs])[0]
                    
                    pmong_prob = get_prediction_matches_no_threshold(pmong_model_predictions,ground_truth_object_id)
                    among_prob = get_prediction_matches_no_threshold(among_model_predictions,ground_truth_object_id)
                    print(pmong_prob, among_prob, padded_ground_truth_bbx)

                    #pmong=cv2.cvtColor(pmong, cv2.COLOR_BGR2RGB)
                    pmong=None
            
                    acc = float(pmong_prob > among_prob)
                    im_ecc_accuracies.append(acc)
            im_ecc_accuracies = np.array(im_ecc_accuracies)
            if(ecc==0):
                mean = im_ecc_accuracies[0]
                std = np.nan
            else:
                mean = np.nanmean(im_ecc_accuracies)
                std = np.nanstd(im_ecc_accuracies)
            accuracies[model_name][im][ecc] = {'mean': mean,
                                               'std': std,
                                               'raw': im_ecc_accuracies}
            mn = accuracies[model_name][im][ecc]['mean']
            print(f'accuracy for model {model_name} for {im} at ecc {ecc} is {mn}')
            #print(accuracies)
    pkl_file = f'{base_dir}/machine_psychophysics_accuracies_full_sdt_model{model_name}_catmatch_{category_match_bool}.pickle'
    file = open(pkl_file,'wb')
    pickle.dump(accuracies[model_name],file)
    file.close()
                


In [12]:
#add this model's evlation to previous ones
import pickle
pkl_file = f'/{base_dir}/machine_psychophysics_accuracies_full_sdt_catmatch_{category_match_bool}.pickle'
file = open(pkl_file,'wb')
pickle.dump(accuracies,file)
file.close()